# California Wildfires
### Donated by: Casey Graff (graffc@uci.edu)

* ### Task: 2D Classificaiton (similiar to image segmentation)
* ### # of Instances: 10k train / 5k test (more upon request)
* ### Data
    * #### Input: 3D Image (# Channels, Height, Width)
    * #### Output: 2D Classification Probabilites (Height, Width)

## Description

This challenge dataset focuses on the spatiotemporal prediction problem of forecasting how wildfires will spread in 12 or 24 hour periods. 

**Active fires** are observed using the VIIRS (Visible Infrared Imaging Radiometer Suite) mounted on the Suomi National Polar-orbiting Partnership (NPP) satellite. 

These fires are influenced by **land cover**, **topography**, and **weather** (among others not captured in this dataset). 

All data sources have been rescaled to **375m / pixel**. Each image contains 30 x 30 pixels  for an effective real **area of 11.25km**.

## Data
### I. VIIRS Fire Detections
* 5 layers / time steps (T = 0, -12, -24, -36, -48 hours)
* 2 targets (T = +12 hours, +24 hours)

### II. Land Cover (LANDFIRE)
* 17 layers (3 topographic, 10 vegetation, 4 fuel/canopy)

### III. Meteorology (Rapid Refresh)
* 2 time steps (T = 0, +12 hours) x 5 variables (temperature, humidity, etc.)

### Imports + Setup

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
FIG_REG_WIDTH = 7
FIG_REG_ASPECT_RATIO = 1.75

def set_fig_settings(fig_size=(32,10), font_size=16, font_scale=1.6):       
    plt.rcParams['figure.figsize'] = fig_size
    plt.rcParams['mathtext.fontset'] = 'stix'
    plt.rcParams["legend.framealpha"] = 0

    font = {'weight' : 'normal', 'size'   : font_size}

    plt.rc('font', **font)

### Data Loading

In [ ]:
DATASET_PATH = 'data/uci_ml_hackathon_fire_dataset_2012-05-09_2013-01-01_10k_train.hdf5'

with h5py.File(DATASET_PATH, 'r') as f:
    train_data = {}
    for k in list(f):
        train_data[k] = f[k][:]

In [ ]:
print(train_data.keys())

In [ ]:
large_fire_inds = np.where(
    (np.sum(train_data['observed'][:,0],axis=(1,2)) > 50) & 
    (np.sum(train_data['observed'][:,1],axis=(1,2)) > 50) & 
    (np.sum(train_data['observed'][:,2],axis=(1,2)) > 50) & 
    (np.sum(train_data['observed'][:,3],axis=(1,2)) > 50) & 
    (np.sum(train_data['observed'][:,4],axis=(1,2)) > 50) & 
    (np.sum(train_data['target'][:,0],axis=(1,2)) > 50) 
)[0]

In [ ]:
TRAINING_POINT = large_fire_inds[0]

## I. VIIRS Detections (Observed + Target)

### Source: https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/viirs-i-band-active-fire-data

### Resolution: ~375m -> 375m

In [ ]:
print('Shape:', train_data['observed'].shape, '= (# of Instances, # of Timesteps/Lags, Width, Height)')

In [ ]:
set_fig_settings((FIG_REG_WIDTH*2,FIG_REG_WIDTH*1.25))
fig = plt.figure()

lat = train_data['latitude'][TRAINING_POINT]
lon = train_data['longitude'][TRAINING_POINT]
datetime = pd.to_datetime(train_data['datetime'][TRAINING_POINT])

fig.suptitle(f'Lat: {lat:.2f}, Lon: {lon:.2f}, Datetime: {datetime}')

# Plot X detections
for i in range(5):
    plt.subplot(2,5,i+1)
    plt.title(f'{-12 * (4-i)} hours')
    plt.imshow(train_data['observed'][TRAINING_POINT][4-i])
    plt.axis('off')

# Plt Y detections
for i in range(2):
    plt.subplot(2,5,i+5+1)
    plt.title(f'+{12 * (i+1)} hours')
    plt.imshow(train_data['target'][TRAINING_POINT][i])
    plt.axis('off')
    
plt.tight_layout()

## II. Land Cover

### Source: https://www.landfire.gov/
### Resolution: 30m -> 375m

#### Layers
* 0: Aspect 
* 1: Canopy Bult Density
* 2: Canopy Base Height
* 3: Canopy Cover
* 4: Canopy Height
* 5: Elevelation
* 6 to 15: Vegetation (Fractional Veg Class per layer)
* 16: Slope

#### Vegetation Layers
* 6: No Data
* 7: Sparse
* 8: Tree
* 9: Shrub
* 10: Herb
* 11: Water
* 12: Barren
* 13: Developed
* 14: Snow-Ice
* 15: Agriculture

In [ ]:
LAND_COVER_LAYER_NAME_TO_IND = {'ASP': 0, 'CBD': 1, 'CBH': 2, 'CC': 3, 'CH': 4, 'DEM': 5, 'EVT': 6, 'SLP': 16}
VEGETATION_NAME_TO_IND = {'Nodata': 0, 'Sparse': 1, 'Tree': 2, 'Shrub': 3, 'Herb': 4, 'Water': 5, 'Barren': 6, 'Developed': 7, 'Snow-Ice': 8, 'Agriculture': 9}

TOPO_NAMES = ['ASP', 'SLP', 'DEM']
VEG_NAME = 'EVT'
FUEL_NAMES = ['CBD', 'CBH', 'CC', 'CH']

In [ ]:
print('Shape:', train_data['land_cover'].shape, '= (# of Instances, # of Layers, Width, Height)')

In [ ]:
set_fig_settings((FIG_REG_WIDTH*1.5,FIG_REG_WIDTH*.65))
fig = plt.figure()

fig.suptitle('Topography')
for i, name in enumerate(TOPO_NAMES):
    plt.subplot(1,len(TOPO_NAMES),i+1)
    plt.imshow(train_data['land_cover'][TRAINING_POINT][LAND_COVER_LAYER_NAME_TO_IND[name]])
    plt.title(name)
    plt.axis('off')
    
plt.tight_layout()

In [ ]:
set_fig_settings((FIG_REG_WIDTH*2,FIG_REG_WIDTH*.65))
fig = plt.figure()

fig.suptitle('Fuel')
for i, name in enumerate(FUEL_NAMES):
    plt.subplot(1,len(FUEL_NAMES),i+1)
    plt.imshow(train_data['land_cover'][TRAINING_POINT][LAND_COVER_LAYER_NAME_TO_IND[name]])
    plt.title(name)
    plt.axis('off')

plt.tight_layout()

In [ ]:
set_fig_settings((FIG_REG_WIDTH*2,FIG_REG_WIDTH*1.25))
fig = plt.figure()

fig.suptitle('Vegetation')
for i, (name, ind) in enumerate(VEGETATION_NAME_TO_IND.items()):
    plt.subplot(2,len(VEGETATION_NAME_TO_IND)//2,i+1)
    plt.imshow(train_data['land_cover'][TRAINING_POINT][LAND_COVER_LAYER_NAME_TO_IND[VEG_NAME] + ind])
    plt.title(name)
    plt.axis('off')

plt.tight_layout()

## III. Weather

### Source: https://rapidrefresh.noaa.gov/
###  Resolution 13km -> 375m

#### Timesteps
* 0: T = 0 hours (closest measurement to observed VIIRS detections)
* 1: T = 12 hours (closest measurement to taget VIIRS detections)

#### Variables
* 0: Temperature @ 2m (Kelvin)
* 1: Relative Humidity @ 2m (%)
* 2: U Wind Component @ 10m (m s**-1)
* 3: V Wind Component @ 10m (m s**-1)
* 4: Precipitation Rate (kg m**-2 s**-1)


In [ ]:
METEOROLOGY_NAME_TO_IND = {'Temp': 0, 'Rel. Humid.': 1, 'U Wind Comp.': 2, 'V Wind Comp.': 3, 'Precip. Rate': 4}

In [ ]:
print('Shape:', train_data['meteorology'].shape, '= (# of Instances, Timesteps, # of Variables, Width, Height)')

In [ ]:
set_fig_settings((FIG_REG_WIDTH*2,FIG_REG_WIDTH*.65))
fig = plt.figure()

fig.suptitle('Vegetation')
for i, (name, ind) in enumerate(METEOROLOGY_NAME_TO_IND.items()):
    plt.subplot(1,len(METEOROLOGY_NAME_TO_IND),i+1)
    plt.imshow(train_data['meteorology'][TRAINING_POINT][0][ind])
    plt.scatter([15], [15], c='red')
    plt.title(name)
    plt.axis('off')

plt.tight_layout()

## Task / Prediction

In [ ]:
import scipy.ndimage

In [ ]:
def persistence_model(x):
    return scipy.ndimage.gaussian_filter(x, 1.7, output=np.float32)

def compute_mse(y, y_hat):
    return np.mean((y-y_hat)**2)

In [ ]:
x = train_data['observed'][TRAINING_POINT,0]
y = train_data['target'][TRAINING_POINT,0]
y_pred = persistence_model(x)

In [ ]:
plt.subplot(1,3,1)
plt.imshow(x)
plt.title('X (T = 0 hours)')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(y_pred)
plt.title('$\hat{Y}$ (T = +12 hours)')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(y)
plt.title('Y (T = +12 hours)')
plt.axis('off')

In [ ]:
y_hats = [persistence_model(train_data['observed'][i,0]) for i in range(len(train_data['observed']))]

In [ ]:
mse_12hour = compute_mse(train_data['target'][:,0], y_hats)
mse_24hour = compute_mse(train_data['target'][:,1], y_hats)

print(f'MSE: 12 Hour Target = {mse_12hour:.4f}, 24 Hour Target = {mse_24hour:.4f}')